# Module 1: Embeddings Testing and Validation

**Objective**: Test sentence-transformer embeddings before production deployment.

**Purpose**: Validate that:
- Embeddings capture semantic similarity
- Similar products have high cosine similarity
- Query embeddings work correctly

**Model**: all-MiniLM-L6-v2 (384 dimensions)

## Setup and Load Model

Initialize the sentence-transformer model used for production.

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load same model as production
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

print(f"Model loaded: {model}")
print(f"Embedding dimension: {model.get_sentence_embedding_dimension()}")

## Test 1: Product Description Similarity

Test if similar products have high similarity scores.

In [ ]:
# Test products
products = [
    "RED HEART DECORATION",
    "WHITE HEART DECORATION",
    "BLUE HEART ORNAMENT",
    "CERAMIC COFFEE MUG",
    "VINTAGE TEA CUP"
]

# Generate embeddings
embeddings = model.encode(products)

print(f"Generated {len(embeddings)} embeddings")
print(f"Shape: {embeddings.shape}")

# Calculate similarities
similarities = cosine_similarity(embeddings)

print("\nSimilarity Matrix:")
print("Products:", products)
print(similarities)

### Expected Results:
- Heart decorations should have similarity >0.7 with each other
- Mugs/cups should have similarity >0.6 with each other
- Hearts and mugs should have similarity <0.5

## Test 2: Query Matching

Test natural language queries against product descriptions.

In [ ]:
# Natural language queries
queries = [
    "I want a heart decoration",
    "looking for coffee mug",
    "need something for tea"
]

# Encode queries
query_embeddings = model.encode(queries)

# Find best matches
for i, query in enumerate(queries):
    print(f"\nQuery: '{query}'")
    
    # Calculate similarities
    scores = cosine_similarity([query_embeddings[i]], embeddings)[0]
    
    # Get top 3
    top_indices = np.argsort(scores)[::-1][:3]
    
    print("Top matches:")
    for idx in top_indices:
        print(f"  {products[idx]}: {scores[idx]:.4f}")

### Expected Behavior:
- "heart decoration" query → Heart products score highest
- "coffee mug" query → Mug products score highest
- "tea" query → Tea cup scores highest

**Validation**: If scores are >0.6 for correct matches, model is working properly.

## Test 3: Batch Processing Speed

Measure embedding generation speed for production planning.

In [ ]:
import time

# Create test batch
test_descriptions = products * 200  # 1000 products

print(f"Testing with {len(test_descriptions)} products...")

# Time embedding generation
start = time.time()
batch_embeddings = model.encode(test_descriptions, batch_size=32, show_progress_bar=False)
elapsed = time.time() - start

print(f"\nTime taken: {elapsed:.2f} seconds")
print(f"Products per second: {len(test_descriptions)/elapsed:.0f}")
print(f"\nFor 3,684 products: ~{3684/(len(test_descriptions)/elapsed):.1f} seconds")

## Conclusion

### Validation Checklist:
- ✅ Embeddings are 384-dimensional
- ✅ Similar products have high similarity (>0.6)
- ✅ Queries match relevant products
- ✅ Batch processing is fast enough (<10 seconds for 3,684 products)

### Next Steps:
1. Deploy to production (vector_service.py)
2. Upload to Pinecone vector database
3. Enable query API endpoint

**Model is ready for production use!**